# 1. Enviornment setup

In [2]:
import os
import cv2
import pandas as pd
import numpy as np
import csv
import torch
import torchvision.models as models
from torch import nn, optim
# import timm 

# multi-threading
import concurrent.futures

# Use half of cpu for multi-threading tasks
# Consider use gpu if CUDA is available
num_workers = os.cpu_count() / 2
print("Number of CPU will be used:", num_workers)

weight_data = 'Data/final_mapping_original.csv'


Number of CPU will be used: 8.0


# 2. Pre-processing

### 2.1 Filtering out background

Filter out all background and leave a pure pig depth image to process

In [3]:
def img_filter(file_addr):
    
    # Load image
    image = cv2.imread(file_addr)

    # Adjust GMM parameters
    backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=False)
    fgMask = backSub.apply(image)

    # Refine the foreground mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgMask = cv2.erode(fgMask, kernel, iterations=2)
    fgMask = cv2.dilate(fgMask, kernel, iterations=2)

    # Convert image to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define color range for the pig color
    lower_color = np.array([110, 255, 254])
    upper_color = np.array([255, 255, 255])
    color_mask = cv2.inRange(hsv, lower_color, upper_color)

    # Combine color mask with foreground mask
    fgMask = cv2.bitwise_and(fgMask, color_mask)

    # Find contours from the mask
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour is the pig and create a mask for it
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        refined_mask = np.zeros_like(fgMask)
        cv2.fillPoly(refined_mask, [largest_contour], 255)
        fgMask = refined_mask

    # Extract the foreground using the refined mask
    foreground = cv2.bitwise_and(image, image, mask=fgMask)
    
    return foreground

Test the filter result

In [4]:
# eg_img = 'Data/Week2/20211002/20211002_3342/_Depth_4780.png' # Good example
# eg_img = 'Data/Week2/20211002/20211002_3342/_Depth_3223.png' # Good example
# eg_img = 'Data/Week2/20211003/20211003_3342a/_Depth_4489.png' # not that bad example
eg_img = 'Data/Week1/20210922/20210922_3342/_Depth_5395.png' # Bad example
foreground = img_filter(eg_img)

# image = cv2.imread(eg_img)
# print("Total number of pixels after filtering:", image.shape[0] * image.shape[1])
# cv2.imshow('Foreground', foreground)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

### 2.2 Count pixels

Count the pixels in the image

In [5]:
def get_pixel_map(img):

    # Convert the foreground to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Flatten the image
    pixels = img.reshape(-1, img.shape[-1])

    # Count unique colors
    unique_colors, counts = np.unique(pixels[pixels.sum(axis=1) != 0], axis=0, return_counts=True)

    # Create a color map
    color_map = {tuple(color): count for color, count in zip(unique_colors, counts)}
    
    return color_map

Test counting pixels

In [ ]:
foreground = img_filter(eg_img)
color_map = get_pixel_map(foreground)
print("Total number of pixels in map:", sum(color_map.values()))
print("Color Map:", color_map)

Total number of pixels in map: 51501
Color Map: {(0, 0, 255): 841, (0, 1, 255): 671, (0, 2, 254): 285, (0, 2, 255): 629, (0, 3, 255): 786, (0, 4, 255): 828, (0, 5, 254): 333, (0, 5, 255): 700, (0, 6, 255): 630, (0, 7, 255): 1000, (0, 8, 255): 760, (0, 9, 254): 416, (0, 9, 255): 726, (0, 10, 255): 692, (0, 11, 255): 758, (0, 12, 254): 333, (0, 12, 255): 351, (0, 13, 255): 989, (0, 14, 254): 281, (0, 14, 255): 553, (0, 15, 255): 846, (0, 16, 255): 912, (0, 17, 255): 679, (0, 18, 255): 1006, (0, 19, 254): 305, (0, 19, 255): 679, (0, 20, 255): 733, (0, 21, 254): 370, (0, 21, 255): 442, (0, 22, 254): 409, (0, 22, 255): 492, (0, 23, 255): 839, (0, 24, 254): 452, (0, 24, 255): 480, (0, 25, 255): 483, (0, 26, 255): 892, (0, 27, 255): 831, (0, 28, 255): 867, (0, 29, 255): 816, (0, 30, 254): 358, (0, 30, 255): 703, (0, 31, 254): 345, (0, 31, 255): 651, (0, 32, 255): 788, (0, 33, 255): 841, (0, 34, 255): 852, (0, 35, 255): 843, (0, 36, 255): 865, (0, 37, 254): 256, (0, 37, 255): 719, (0, 38, 254)

### 2.3 Mapping weight w/ pixel map
Read weight w/ the pixel map  
Use weight, image_path, color_map to access the data

In [6]:
def img_to_data(row):
    image_path = 'Data/' + row[0]
    base_name, _ = os.path.splitext(image_path)
    image_path = f'{base_name}.png' # Change from jpg to png
    # print(image_path)
    
    # if using whole image files, use this one
    if not os.path.exists(image_path): 
        # print(f"Image not found: {image_path}")
        return
    
    # Get the foreground by calling img_filter
    foreground = img_filter(image_path)
    
    # Get the pixel map by calling get_pixel_map
    color_map = get_pixel_map(foreground)
    
    return {
                'weight': row[3],
                'image_path': image_path,
                'color_map': color_map
            }
            
def get_data():
    with open(weight_data, mode='r', newline='') as infile:
        reader = csv.reader(infile)

        # Create a new list to hold the combined data
        combined_data = []
        headers = next(reader)
        
        # multi-threading data process
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
            # Start a set of tasks and mark each future with its URL
            future_to_task = {executor.submit(img_to_data, arg): arg for arg in reader}
            for future in concurrent.futures.as_completed(future_to_task):
                arg = future_to_task[future]
                try:
                    data = future.result()
                    combined_data.append(data)
                except Exception as exc:
                    print(f'{arg} generated an exception: {exc}')
                else:
                    print(f'{arg} page is {len(data)} bytes')
                


    # Convert the combined data to a DataFrame
    combined_df = pd.DataFrame(combined_data)

    # Display the combined DataFrame
    # print(combined_df)
    return combined_df

Test integrated color map and weight

In [7]:
data = get_data()

print(data)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/final_mapping_original.csv'

# 3. Model training

TODO HERE